In [46]:
from main import get_custom_features, add_classic_indicators
import pandas as pd

eth_usdt = pd.read_pickle('usdt_data/ETHUSDT.pickle').astype(float)
eth_usdt['returns'] = eth_usdt['close'].pct_change()
eth_usdt = eth_usdt.dropna()

In [48]:
eth_usdt = add_classic_indicators(eth_usdt)

In [49]:
eth_usdt = get_custom_features(eth_usdt)

In [87]:
eth_usdt

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore,returns,macd_24_52,macd_12_26,ema5,sma5,ema8,sma8,ema12,sma12,ema16,sma16,ema20,sma20,ema26,sma26,adx7,rsi7,stochrsi7,willR7,cci7,adx10,rsi10,stochrsi10,willR10,cci10,adx14,rsi14,stochrsi14,willR14,...,alpha30,alpha32,alpha33,alpha34,alpha35,alpha36,alpha37,alpha38,alpha40,alpha41,alpha42,alpha43,alpha44,alpha45,alpha47,alpha49,alpha50,alpha51,alpha52,alpha53,alpha54,alpha55,alpha60,alpha61,alpha62,alpha64,alpha65,alpha68,alpha74,alpha75,alpha78,alpha81,alpha83,alpha84,alpha85,alpha86,alpha94,alpha95,alpha99,alpha101
2018-01-01,727.01,732.00,716.80,717.97,2305.97086,1.514772e+12,1.675753e+06,2875.0,1035.33513,7.532118e+05,0.0,-0.013262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.915131,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,NaN
2018-01-01,717.67,725.75,717.59,724.05,2166.45725,1.514776e+12,1.564151e+06,2957.0,1179.82843,8.519421e+05,0.0,0.008468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,0.114902,NaN,NaN,NaN,NaN,NaN,NaN,0.393467,0.272727,NaN,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,NaN,-7.573969e-01,NaN,NaN,0,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0.781767
2018-01-01,723.95,737.99,722.70,734.50,2160.90450,1.514779e+12,1.582200e+06,3647.0,1095.63271,8.014701e+05,0.0,0.014433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,0.051690,NaN,NaN,NaN,NaN,NaN,NaN,0.519986,0.196330,NaN,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,NaN,-7.178908e-01,NaN,NaN,0,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0.689948
2018-01-01,734.99,744.98,730.01,744.82,2335.33705,1.514783e+12,1.724788e+06,3512.0,1313.03081,9.704303e+05,0.0,0.014050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,0.059845,NaN,NaN,NaN,NaN,NaN,NaN,-1.242984,0.161041,NaN,NaN,NaN,NaN,-1.000000,NaN,-1.000000,NaN,NaN,-9.257288e-01,NaN,NaN,0,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0.656603
2018-01-01,744.78,750.00,742.08,748.99,2289.19266,1.514786e+12,1.711600e+06,3530.0,1398.97277,1.046253e+06,0.0,0.005599,NaN,NaN,737.326049,734.066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,0.192278,0.470411,NaN,NaN,NaN,NaN,NaN,-0.433010,0.284140,NaN,-0.544334,NaN,NaN,-1.000000,NaN,-1.000000,NaN,NaN,-8.482284e-01,NaN,NaN,0,0,0,0,0,0,0,NaN,0,NaN,NaN,NaN,0,NaN,0,0,0.531499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-24,1625.59,1632.12,1622.22,1627.34,9920.66530,1.674565e+12,1.614472e+07,16233.0,4971.08080,8.090093e+06,0.0,0.001070,-0.689020,-0.951439,1627.037892,1625.738,1628.350859,1629.86500,1629.295415,1632.289167,1629.727572,1631.558750,1629.974278,1630.8745,1630.219026,1628.914231,33.493091,46.009402,0.648186,-46.376812,-22.558140,31.904394,46.946443,0.512186,-50.928382,-48.990936,29.387345,47.786550,0.493632,-52.161064,...,0.021851,-0.000209,0.427548,0.783989,3080.0,2.962383,1.162161,-0.166677,0.021062,0.344971,0.394062,70.0,-0.266843,-0.379215,0.019429,-1.740000,-0.656358,-1.000000,-34.763702,0.840712,-5.143969e-01,0.308044,-0.000051,1,-1,-1,0,0,-1,0,0.715802,0,0.077992,0.929597,0.991877,0,-0.310630,1,-1,0.176750
2023-01-24,1627.34,1630.76,1612.83,1617.14,18600.23910,1.674569e+12,3.017264e+07,25777.0,8136.18780,1.319919e+07,0.0,-0.006268,-1.019183,-1.482149,1623.738595,1622.204,1625.859557,162

In [91]:
test = eth_usdt['returns'].copy()
test[test>0.01] = 1
test[test>0.03] = 2
test[test<=0.01] = 0

In [92]:
test

2018-01-01    0.0
2018-01-01    0.0
2018-01-01    1.0
2018-01-01    1.0
2018-01-01    0.0
             ... 
2023-01-24    0.0
2023-01-24    0.0
2023-01-24    0.0
2023-01-24    0.0
2023-01-24    0.0
Name: returns, Length: 44271, dtype: float64

In [64]:
import numpy as np

data_fourier = pd.concat([pd.Series(eth_usdt.index.values), eth_usdt['close'].reset_index().iloc[:,1:]], axis=1)
data_fourier.columns = ['date','price']
close_fft = np.fft.fft(np.asarray(data_fourier['price'].tolist()))
fft_df = pd.DataFrame({'fft':close_fft})
fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))
fft_list = np.asarray(fft_df['fft'].tolist())
for num_ in [2, 7, 15, 100]:
    fft_list_m10= np.copy(fft_list); fft_list_m10[num_:-num_]=0
    data_fourier[f'Fourier {num_}'] = fft_list_m10